In [1]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
train_df = pd.read_csv("../../input/feedback-prize-english-language-learning/train.csv")
train_df.head(2)

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5


In [3]:
model_path_list = [
#     ('../../14_Baseline4/exp/result/14_v1_01/oof_df.csv',1), # deberta-v3-base
#     ('../../14_Baseline4/exp/result/14_v1_07/oof_df.csv',1), # deberta-v3-base, 10folds    
#     ('../../14_Baseline4/exp/result/14_v1_08/oof_df.csv',1), # deberta-base
#     ('../../14_Baseline4/exp/result/14_v1_09/oof_df.csv',1), # roberta-base
#     ('../../14_Baseline4/exp/result/14_v1_10/oof_df.csv',1), # deberta-v3-large
#     ('../../14_Baseline4/exp/result/14_v1_11/oof_df.csv',1), # xlm-roberta-base
#     ('../../14_Baseline4/exp/result/14_v1_12/oof_df.csv',1), # deberta-large
#     ('../../14_Baseline4/exp/result/14_v1_13/oof_df.csv',1), # roberta-large
#     ('../../14_Baseline4/exp/result/14_v1_14/oof_df.csv',1), # xlm-roberta-large
#     ('../../14_Baseline4/exp/result/14_v1_15/oof_df.csv',1), # deberta-v2-xlarge
#     ('../../14_Baseline4/exp/result/14_v1_16/oof_df.csv',1), # deberta-xlarge
    
#     ('../../14_Baseline4/exp/result/14_v2_01/oof_df.csv',1), # deberta-v3-base, seed200
#     ('../../14_Baseline4/exp/result/14_v2_02/oof_df.csv',1), # deberta-v3-large, seed200
#     ('../../14_Baseline4/exp/result/14_v2_03/oof_df.csv',1), # deberta-base, seed200
    
    #('../../17_Pseudo3/exp/result/17_v1_01/oof_df.csv',1), # deberta-v3-base
    ('../../17_Pseudo3/exp/result/17_v1_02/oof_df.csv',1), # deberta-v3-base
    ('../../17_Pseudo3/exp/result/17_v1_03/oof_df.csv',1), # deberta-v3-large
    ('../../17_Pseudo3/exp/result/17_v1_04/oof_df.csv',1), # deberta-v3-base, seed200
    ('../../17_Pseudo3/exp/result/17_v1_05/oof_df.csv',1), # deberta-base
    ('../../17_Pseudo3/exp/result/17_v1_06/oof_df.csv',1), # deberta-large
]

oof_df_list = [
    pd.read_csv(model_path) for model_path,_ in model_path_list
]
weights_list = [
    w for _,w in model_path_list
]
weights_list = [w/sum(weights_list) for w in weights_list]

for oof_df in oof_df_list:
    oof_df = train_df[['text_id']].merge(oof_df, how='left', on='text_id')
    
num_models = len(model_path_list)

TARGET_COLS = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']

preds = 0
for oof_df, w in zip(oof_df_list, weights_list):
    preds += oof_df[TARGET_COLS].values * w

oofs = []
for oof_df in oof_df_list:
    oofs.append(oof_df[TARGET_COLS].values)

In [4]:
oof_df = oof_df_list[0].copy()
for i,col in enumerate(TARGET_COLS):
    oof_df[col] = preds[:,i]

In [5]:
import numpy as np

def calc_metric(pred, gt):
    '''
    pred : (num_data, num_labels)
    gt : (num_data, num_labels)
    '''
    score = np.sqrt(np.mean((pred - gt)**2, axis=0))
    score = score.mean()
    return score

In [6]:
score = calc_metric(pred=oof_df[TARGET_COLS].values, gt=train_df[TARGET_COLS].values)
print('CV={:.4f}'.format(score))

CV=0.4456


# Optimize class-wise

In [7]:
ensemble_predictions=np.stack(oofs)
ensemble_predictions.shape

(5, 3911, 6)

In [8]:
from skopt import gp_minimize

def run_optimize(i):
    target_col = TARGET_COLS[i]
    print("*"*50)
    print(target_col)
    print("*"*50)
    def ensemble_score(weights,return_pred=False):
        weights=np.array(weights)
        weights=weights.reshape(-1,1,1)/weights.sum()
        p=weights.reshape(-1,1,1)*ensemble_predictions[:,:,i,None]
        p=p.sum(0)
        score=calc_metric(p, train_df[[target_col]])
        if return_pred:
            return score,p
        else:
            return score   
    results = gp_minimize(ensemble_score, np.array([[0.1,1] for i in range(len(ensemble_predictions))]),
                          verbose=True,random_state=2022)
    best_weights=np.array(results['x'])/sum(results['x'])
    print(best_weights)
    # smaller is better for this metric
    score, ensemble_pred=ensemble_score(best_weights,True)
    print("score={:.4f}", score)
    return score, best_weights

In [9]:
scores = {}
best_weights = {}
for i,target_col in enumerate(TARGET_COLS):
    score, weights = run_optimize(i)
    scores[target_col] = score
    best_weights[target_col] = weights

**************************************************
cohesion
**************************************************
Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0034
Function value obtained: 0.4745
Current minimum: 0.4745
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0025
Function value obtained: 0.4745
Current minimum: 0.4745
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0022
Function value obtained: 0.4743
Current minimum: 0.4743
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0023
Function value obtained: 0.4744
Current minimum: 0.4743
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.

Iteration No: 41 ended. Search finished for the next optimal point.
Time taken: 0.6794
Function value obtained: 0.4746
Current minimum: 0.4737
Iteration No: 42 started. Searching for the next optimal point.
Iteration No: 42 ended. Search finished for the next optimal point.
Time taken: 0.6420
Function value obtained: 0.4737
Current minimum: 0.4737
Iteration No: 43 started. Searching for the next optimal point.
Iteration No: 43 ended. Search finished for the next optimal point.
Time taken: 0.6859
Function value obtained: 0.4747
Current minimum: 0.4737
Iteration No: 44 started. Searching for the next optimal point.
Iteration No: 44 ended. Search finished for the next optimal point.
Time taken: 0.5753
Function value obtained: 0.4742
Current minimum: 0.4737
Iteration No: 45 started. Searching for the next optimal point.
Iteration No: 45 ended. Search finished for the next optimal point.
Time taken: 0.5558
Function value obtained: 0.4737
Current minimum: 0.4737
Iteration No: 46 started. Sea

Iteration No: 81 ended. Search finished for the next optimal point.
Time taken: 0.9116
Function value obtained: 0.4737
Current minimum: 0.4737
Iteration No: 82 started. Searching for the next optimal point.
Iteration No: 82 ended. Search finished for the next optimal point.
Time taken: 0.9515
Function value obtained: 0.4751
Current minimum: 0.4737
Iteration No: 83 started. Searching for the next optimal point.
Iteration No: 83 ended. Search finished for the next optimal point.
Time taken: 0.8997
Function value obtained: 0.4750
Current minimum: 0.4737
Iteration No: 84 started. Searching for the next optimal point.
Iteration No: 84 ended. Search finished for the next optimal point.
Time taken: 1.1186
Function value obtained: 0.4741
Current minimum: 0.4737
Iteration No: 85 started. Searching for the next optimal point.
Iteration No: 85 ended. Search finished for the next optimal point.
Time taken: 0.8183
Function value obtained: 0.4737
Current minimum: 0.4737
Iteration No: 86 started. Sea

Iteration No: 21 ended. Search finished for the next optimal point.
Time taken: 0.3893
Function value obtained: 0.4399
Current minimum: 0.4393
Iteration No: 22 started. Searching for the next optimal point.
Iteration No: 22 ended. Search finished for the next optimal point.
Time taken: 0.4021
Function value obtained: 0.4392
Current minimum: 0.4392
Iteration No: 23 started. Searching for the next optimal point.
Iteration No: 23 ended. Search finished for the next optimal point.
Time taken: 0.3142
Function value obtained: 0.4397
Current minimum: 0.4392
Iteration No: 24 started. Searching for the next optimal point.
Iteration No: 24 ended. Search finished for the next optimal point.
Time taken: 0.3732
Function value obtained: 0.4394
Current minimum: 0.4392
Iteration No: 25 started. Searching for the next optimal point.
Iteration No: 25 ended. Search finished for the next optimal point.
Time taken: 0.3915
Function value obtained: 0.4392
Current minimum: 0.4392
Iteration No: 26 started. Sea

Iteration No: 61 ended. Search finished for the next optimal point.
Time taken: 0.7601
Function value obtained: 0.4396
Current minimum: 0.4392
Iteration No: 62 started. Searching for the next optimal point.
Iteration No: 62 ended. Search finished for the next optimal point.
Time taken: 0.7289
Function value obtained: 0.4392
Current minimum: 0.4392
Iteration No: 63 started. Searching for the next optimal point.
Iteration No: 63 ended. Search finished for the next optimal point.
Time taken: 0.7386
Function value obtained: 0.4392
Current minimum: 0.4392
Iteration No: 64 started. Searching for the next optimal point.
Iteration No: 64 ended. Search finished for the next optimal point.
Time taken: 0.9106
Function value obtained: 0.4398
Current minimum: 0.4392
Iteration No: 65 started. Searching for the next optimal point.
Iteration No: 65 ended. Search finished for the next optimal point.
Time taken: 0.7677
Function value obtained: 0.4392
Current minimum: 0.4392
Iteration No: 66 started. Sea

...

Iteration No: 10 ended. Evaluation done at random point.
Time taken: 0.3296
Function value obtained: 0.4657
Current minimum: 0.4641
Iteration No: 11 started. Searching for the next optimal point.
Iteration No: 11 ended. Search finished for the next optimal point.
Time taken: 0.3626
Function value obtained: 0.4640
Current minimum: 0.4640
Iteration No: 12 started. Searching for the next optimal point.
Iteration No: 12 ended. Search finished for the next optimal point.
Time taken: 0.3135
Function value obtained: 0.4653
Current minimum: 0.4640
Iteration No: 13 started. Searching for the next optimal point.
Iteration No: 13 ended. Search finished for the next optimal point.
Time taken: 0.3286
Function value obtained: 0.4638
Current minimum: 0.4638
Iteration No: 14 started. Searching for the next optimal point.
Iteration No: 14 ended. Search finished for the next optimal point.
Time taken: 0.3651
Function value obtained: 0.4654
Current minimum: 0.4638
Iteration No: 15 started. Searching for 

Iteration No: 50 ended. Search finished for the next optimal point.
Time taken: 0.6052
Function value obtained: 0.4647
Current minimum: 0.4632
Iteration No: 51 started. Searching for the next optimal point.
Iteration No: 51 ended. Search finished for the next optimal point.
Time taken: 0.7930
Function value obtained: 0.4654
Current minimum: 0.4632
Iteration No: 52 started. Searching for the next optimal point.
Iteration No: 52 ended. Search finished for the next optimal point.
Time taken: 0.6860
Function value obtained: 0.4651
Current minimum: 0.4632
Iteration No: 53 started. Searching for the next optimal point.
Iteration No: 53 ended. Search finished for the next optimal point.
Time taken: 0.6267
Function value obtained: 0.4658
Current minimum: 0.4632
Iteration No: 54 started. Searching for the next optimal point.
Iteration No: 54 ended. Search finished for the next optimal point.
Time taken: 0.7006
Function value obtained: 0.4632
Current minimum: 0.4632
Iteration No: 55 started. Sea

Iteration No: 90 ended. Search finished for the next optimal point.
Time taken: 1.0420
Function value obtained: 0.4632
Current minimum: 0.4632
Iteration No: 91 started. Searching for the next optimal point.
Iteration No: 91 ended. Search finished for the next optimal point.
Time taken: 1.2291
Function value obtained: 0.4650
Current minimum: 0.4632
Iteration No: 92 started. Searching for the next optimal point.
Iteration No: 92 ended. Search finished for the next optimal point.
Time taken: 1.1582
Function value obtained: 0.4646
Current minimum: 0.4632
Iteration No: 93 started. Searching for the next optimal point.
Iteration No: 93 ended. Search finished for the next optimal point.
Time taken: 1.1720
Function value obtained: 0.4653
Current minimum: 0.4632
Iteration No: 94 started. Searching for the next optimal point.
Iteration No: 94 ended. Search finished for the next optimal point.
Time taken: 1.2390
Function value obtained: 0.4632
Current minimum: 0.4632
Iteration No: 95 started. Sea

Iteration No: 30 ended. Search finished for the next optimal point.
Time taken: 0.3524
Function value obtained: 0.4384
Current minimum: 0.4374
Iteration No: 31 started. Searching for the next optimal point.
Iteration No: 31 ended. Search finished for the next optimal point.
Time taken: 0.3569
Function value obtained: 0.4374
Current minimum: 0.4374
Iteration No: 32 started. Searching for the next optimal point.
Iteration No: 32 ended. Search finished for the next optimal point.
Time taken: 0.4819
Function value obtained: 0.4385
Current minimum: 0.4374
Iteration No: 33 started. Searching for the next optimal point.
Iteration No: 33 ended. Search finished for the next optimal point.
Time taken: 0.5546
Function value obtained: 0.4383
Current minimum: 0.4374
Iteration No: 34 started. Searching for the next optimal point.
Iteration No: 34 ended. Search finished for the next optimal point.
Time taken: 0.5645
Function value obtained: 0.4392
Current minimum: 0.4374
Iteration No: 35 started. Sea

Iteration No: 70 ended. Search finished for the next optimal point.
Time taken: 0.7122
Function value obtained: 0.4391
Current minimum: 0.4374
Iteration No: 71 started. Searching for the next optimal point.
Iteration No: 71 ended. Search finished for the next optimal point.
Time taken: 0.8011
Function value obtained: 0.4374
Current minimum: 0.4374
Iteration No: 72 started. Searching for the next optimal point.
Iteration No: 72 ended. Search finished for the next optimal point.
Time taken: 0.8269
Function value obtained: 0.4383
Current minimum: 0.4374
Iteration No: 73 started. Searching for the next optimal point.
Iteration No: 73 ended. Search finished for the next optimal point.
Time taken: 0.9557
Function value obtained: 0.4391
Current minimum: 0.4374
Iteration No: 74 started. Searching for the next optimal point.
Iteration No: 74 ended. Search finished for the next optimal point.
Time taken: 0.7388
Function value obtained: 0.4386
Current minimum: 0.4374
Iteration No: 75 started. Sea

In [10]:
best_weights

{'cohesion': array([0.18512471, 0.44006328, 0.06175827, 0.04400633, 0.26904741]),
 'syntax': array([0.40351388, 0.40351388, 0.04035139, 0.04035139, 0.11226946]),
 'vocabulary': array([0.1633802 , 0.3407573 , 0.3407573 , 0.03407573, 0.12102947]),
 'phraseology': array([0.27776016, 0.33813838, 0.1727026 , 0.03381384, 0.17758503]),
 'grammar': array([0.26492302, 0.56544383, 0.05654438, 0.05654438, 0.05654438]),
 'conventions': array([0.46521853, 0.39521591, 0.04652185, 0.04652185, 0.04652185])}

In [11]:
scores

{'cohesion': 0.4737066151889168,
 'syntax': 0.4391862653745737,
 'vocabulary': 0.4061554169416803,
 'phraseology': 0.44845676452597355,
 'grammar': 0.4631713694745503,
 'conventions': 0.43736527257852087}

In [12]:
CV = sum(scores.values()) / len(scores)
print("CV={:.4f}".format(CV))

CV=0.4447


# Error Analysis - Check Corr

In [13]:
cols = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
train_df[cols].corr()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
cohesion,1.000000,0.695459,0.666151,0.690058,0.638689,0.666151
syntax,0.695459,1.000000,0.680562,0.725467,0.709525,0.700025
vocabulary,0.666151,0.680562,1.000000,0.735261,0.654852,0.664292
phraseology,0.690058,0.725467,0.735261,1.000000,0.719746,0.666842
grammar,0.638689,0.709525,0.654852,0.719746,1.000000,0.673301
conventions,0.666151,0.700025,0.664292,0.666842,0.673301,1.000000


In [14]:
cols = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
oof_df[cols].corr()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
cohesion,1.000000,0.947054,0.916545,0.914623,0.826981,0.890427
syntax,0.947054,1.000000,0.940688,0.965179,0.921285,0.931798
vocabulary,0.916545,0.940688,1.000000,0.963551,0.878121,0.895905
phraseology,0.914623,0.965179,0.963551,1.000000,0.945162,0.890967
grammar,0.826981,0.921285,0.878121,0.945162,1.000000,0.859472
conventions,0.890427,0.931798,0.895905,0.890967,0.859472,1.000000
